# 神经网络


## 多维数组运算

### 使用unmpy处理数组

In [2]:
import numpy as np
A = np.array([1,2,3,4])
# 打印数组
print(A) 

[1 2 3 4]


In [ ]:
# 输出数组维度
print(np.ndim(A))

In [ ]:
# 数组形状
print(A.shape)
# 数组第一个维度形状
print(A.shape[0])

### 矩阵的乘法
使用np.dot(A,B)计算数组乘法

In [ ]:
B = np.array([[1,2],
              [3,4]])

C = np.array([[5,6],
              [7,8]])

print(np.dot(B,C))

In [ ]:
X = np.array([1,2])
W = np.array([[1,3,5],
              [2,4,6]])
print(np.dot(X,W))

### 三层神经网络实现

输入层到第一层

输入X, 第一层 $ A^{(1)} $, 权重 $ W^{(1)} $, 偏置 $ B^{(1)} $

$ A^{(1)}= XW^{(1)} + B^{(1)}  $

In [ ]:
# sigmod函数
def sigmod(x):
    return 1 / (1 + np.exp(-x))

In [ ]:

X = np.array([1.0, 0.5])
W1 = np.array([[0.1, 0.3, 0.5],
               [0.2, 0.4, 0.6]])
B1 = np.array([0.1, 0.2, 0.3])

A1 = np.dot(X, W1) + B1
print(A1)
Z1 = sigmod(A1)
print(Z1)

第一层到第二层

In [ ]:
W2 = np.array([[0.1, 0.4],
               [0.2, 0.5],
               [0.3, 0.6]])
B2 = np.array([0.1, 0.2])

A2 = np.dot(Z1,W2) + B2
print(A2)
Z2 = sigmod(A2)
print(Z2)

第二层到输出层

In [ ]:
def identity_func(x):
    return x;

W3 = np.array([[0.1, 0.3],
               [0.2, 0.4]])
B3 = np.array([0.1, 0.2])
A3 = np.dot(Z2,W3) + B3
print(A3)

Y = identity_func(A3)

代码实现小结

In [ ]:
def init_network():
    network = {}
    network['W1'] = np.array([[0.1, 0.3, 0.5],
                              [0.2, 0.4, 0.6]])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['W2'] = np.array([[0.1, 0.4],
                              [0.2, 0.5],
                              [0.3, 0.6]])
    network['b2'] = np.array([0.1, 0.2])
    network['W3'] = np.array([[0.1, 0.3],
                              [0.2, 0.4]])
    network['b3'] = np.array([0.1, 0.2])

    return network

def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x,W1) + b1
    z1 = sigmod(a1)

    a2 = np.dot(z1,W2) + b2
    z2 = sigmod(a2)

    a3 = np.dot(z2,W3) + b3
    y = identity_func(a3)

    return y

network = init_network()
x = np.array([1.0, 0.5])
y = forward(network, x)
print(y)

## 输出层设计

神经网络可以用在分类问题和回归问题上，不过要根据情况改变输出层的激活函数。一般回归问题用恒等函数，分类问题用softmax函数。

### 恒等函数

恒等函数会将输入原样输出

### softmax函数

$ y_k = \frac{exp(a_k)}{\sum_{i=1}^n {exp(a_i)}}$

其中 $ exp(x) $ 表示 $ e^x $

In [ ]:
a = np.array([0.3, 2.9, 4.0])
exp_a = np.exp(a)
print(exp_a)

sum_exp_a = np.sum(exp_a)
print(sum_exp_a)

y = exp_a / sum_exp_a
print(y)

实现softmax函数

In [ ]:
#会出现溢出问题,a比较大时，e的a次幂就会变得相当大
def softmax_1(a):
    exp_a = np.exp(a)
    sym_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

print(softmax_1(100))
print(softmax_1(np.array([1010,1000,900])))

softmax函数改进

$ y_k 
= \frac{e^{a_k}}{\sum_{i=1}^n {e^{a_i}}} 
= \frac{C e^{a_k}}{C \sum_{i=1}^n {e^{a_i}}} 
= \frac{e^{a_k + logC}}{\sum_{i=1}^n {e^{a_i + logC}}} 
= \frac{e^{a_k + C'}}{\sum_{i=1}^n {e^{a_i + C'}}}
= \frac{exp(a_k + C')}{\sum_{i=1}^n {exp(a_i + C')}} $ 

通过上式，发现在进行softmax指数函数运算时，加上或减去某个常数并不会改变计算结果

softmax函数输出是0.0道1.0之间的实数，并且softmax函数的输出值总和是1。正是应为这个性质，我们才可以把softmax函数解释成概率

In [3]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c) #溢出对策
    sum_exp_a = np.sum(exp_a)
    y = exp_a/sum_exp_a
    return y

a = np.array([1010,1000,900])
y = softmax(a)
print(y)
print(np.sum(y))

[9.99954602e-01 4.53978687e-05 1.68883521e-48]
1.0


## 手写数字识别

In [1]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

#第一次调用会下载数据集
(x_train, t_train) , (x_test, t_test) = load_mnist(flatten=True, normalize=False)

#输出各个数据的形状
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [1]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()


#第一次调用会下载数据集
(x_train, t_train) , (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


In [14]:
# 神经网络的推理处理
import sys, os
sys.path.append(os.pardir)  # 为了导入父目录的文件而进行的设定
import numpy as np
import pickle
from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax


def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    #np.dot矩阵乘法,x(784),W1(784,50) W1转置后与x相乘
    a1 = np.dot(x, W1) + b1 #a1(50)
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2#z1(50) W2(50,100)
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3#z2(100) W2(100,10)
    y = softmax(a3)

    return y


x, t = get_data()
network = init_network()
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p= np.argmax(y) # 获取概率最高的元素的索引
    if p == t[i]:
        accuracy_cnt += 1

#批处理
# batch_size = 100
# accuracy_cnt = 0
# for i in range(1, len(x), batch_size): #range(start, end, step)
#     x_batch = x[i:i+batch_size]
#     y_batch = predict(network, x_batch)
#     p= np.argmax(y) 
#     accuracy_cnt += np.sum(p==t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(1

In [ ]:
#